<h1 style='color: blue; font-size: 34px; font-weight: bold;'> Projeto Proposto 
</h1>
<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Este projeto tem o intuito de estudar Redes Neurais Artificiais e suas aplicações em Deep Learning </i>       
</p>  

# <font color='red' style='font-size: 40px;'> Bibliotecas Utilizadas </font>
<hr style='border: 2px solid red;'>

In [1]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.weightstats import ztest
from statsmodels.stats.diagnostic import lilliefors
from statsmodels.tsa import stattools
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import bernoulli, binom, poisson, geom, norm, chi2, f, normaltest, ttest_ind, ttest_rel, wilcoxon, mannwhitneyu, kruskal
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px 
import plotly.graph_objects as go
import importlib
import transition_matrix_estimator
from transition_matrix_estimator import TransitionMatrixLearner
import time
import datetime
import warnings
import category_encoders as ce 
from category_encoders import BinaryEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix

%matplotlib inline
sns.set(style="whitegrid", font_scale=1.2)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x) # Tira os números do formato de Notação Científica
np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# <font color='red' style='font-size: 40px;'> Exemplo Montagem de Target </font>
<hr style='border: 2px solid red;'>

# <font color='green' style='font-size: 30px;'> 1.1) Teste 1 </font>
<hr style='border: 2px solid green;'>

> 1. Testes de Continuidade: Verificação da consistência temporal dos dados por contrato (sem quebras ou lacunas indevidas).

> 2. Criação da Coluna flag_acordo: Identificação de registros com acordos ou renegociações.

> 3. Criação da Coluna over90: Flag mensal de inadimplência grave (atraso superior a 90 dias).

> 4. Criação da Coluna mau: Identificação de contratos irrecuperáveis com base em regras de negócio.

> 5. Construção do Grupo Performing: Definição da amostra elegível para modelagem de PD, com base no status performing.

> 6. Criação da Coluna ever90m12: Indica se o contrato atingiu atraso >90 dias em qualquer mês dos próximos 12 meses (sem considerar acordos).

> 7. Criação da Coluna over90m12: Indica se o contrato está em atraso >90 dias ao final da janela futura de 12 meses (sem considerar acordos).

> 8. Criação da Coluna target: Flag final de inadimplência futura, considerando tanto atraso quanto acordos.

> 9. Congelamento do Snapshot de Contratos: Seleção de uma linha por contrato para representar o momento de avaliação (janela de decisão).

> 10. Curva de Inadimplência (over90) por Mês: Análise temporal do volume de contratos inadimplentes mês a mês (painel e snapshot).

> 11. Curvas de ever90m12 e over90m12 por Mês: Evolução mensal dos indicadores prospectivos de risco (apenas painel).

> 12. Construção da Matriz de Transição: Avaliação da migração de status de crédito entre períodos (ex: performing → inadimplente).

> 13. Construção da Curva de Cura: Medição da recuperação de contratos inadimplentes ao longo do tempo.

### 1.1.1) Entendimento de Conceitos

> 1. data_inicio_contrato

- É a data que o cliente e o banco firmaram acordo, como por exemplo 2014-09-30
- Em muitos casos, existe "carência" até o 1 vencimento, portanto, o contrato existe juridicamente mas não dá para medir performance ainda

> 2. safra:

- Mês de nascimento da performance
- Se a primeira parcela vence em jan/2025, o contrato é marcado como safra de 201501

> 3. dat_ref:

- A cada dat_ref (foto mensal), o contrato é monitorado
- A safra nunca muda, porque é o "carimbo" da geração

In [ ]:
historico  = pd.read_parquet('./data/full_history.parquet')
historico['data_ref'] = pd.to_datetime(historico['data_ref'])
historico .sort_values(by=['id_contrato','data_ref'], inplace=True)

rastro_contratos = pd.read_parquet('./data/rastro_contratos.parquet')
rastro_contratos['data_evento'] = pd.to_datetime(rastro_contratos['data_evento'])
rastro_contratos.sort_values(by=['id_antigo','id_novo', 'data_evento'], inplace=True)

display(historico.head(10))
display(rastro_contratos.head(10))



,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso
0,10000000,2014-09-30,2015-01-01,201501,30
1,10000000,2014-09-30,2015-02-01,201501,15
2,10000000,2014-09-30,2015-03-01,201501,15
3,10000000,2014-09-30,2015-04-01,201501,30
4,10000000,2014-09-30,2015-05-01,201501,60
5,10000000,2014-09-30,2015-06-01,201501,90
6,10000000,2014-09-30,2015-07-01,201501,60
7,10000000,2014-09-30,2015-08-01,201501,90
8,10000000,2014-09-30,2015-09-01,201501,90
9,10000000,2014-09-30,2015-10-01,201501,60


,id_antigo,id_novo,data_evento
1023,10000000,10004583,2015-12-01
368,10000001,10003338,2015-06-01
706,10000002,10003996,2015-09-01
0,10000003,10002500,2015-02-01
255,10000007,10003118,2015-05-01
1366,10000009,10005220,2016-03-01
815,10000010,10004198,2015-10-01
55,10000011,10002680,2015-03-01
816,10000013,10004199,2015-10-01
256,10000015,10003119,2015-05-01


### 1.1.2) Flag de Renegociação   

> 1. Renegociação é quando o cliente não consegue mais horar o contrato original e o banco faz um novo acordo para tentar recuperar o crédito. Exemplo:

- O cliente estava atrasado várias parcelas
- Ao invés do banco contabilizar uma perda efetivamente, o banco oferece uma renegociação:
    - Pode reparcelar o saldo devedor em novas condições de prazos
    - Pode descontar parte da dívida
    - Pode dar carência antes de retomar os pagamentos
- Nesses casos, o ID do contrato geralmente muda e o novo ID carrega a dívida renegociada

In [ ]:
# Eventos para o contrato antigo (renegociado) …
flag_old = (
    rastro_contratos[['id_antigo', 'data_evento_mes']]
    .rename(columns={'id_antigo': 'id_contrato'})
)

# … e para o contrato novo (substituto)
flag_new = (
    rastro_contratos[['id_novo', 'data_evento_mes']]
    .rename(columns={'id_novo': 'id_contrato'})
)

flag_acordo_tb = (
    pd.concat([flag_old, flag_new], ignore_index=True)
    .assign(flag_acordo=1)
    .drop_duplicates()


In [17]:
historico.loc[historico['id_contrato'] == '10000000']

,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso
0,10000000,2014-09-30,2015-01-01,201501,30
1,10000000,2014-09-30,2015-02-01,201501,15
2,10000000,2014-09-30,2015-03-01,201501,15
3,10000000,2014-09-30,2015-04-01,201501,30
4,10000000,2014-09-30,2015-05-01,201501,60
5,10000000,2014-09-30,2015-06-01,201501,90
6,10000000,2014-09-30,2015-07-01,201501,60
7,10000000,2014-09-30,2015-08-01,201501,90
8,10000000,2014-09-30,2015-09-01,201501,90
9,10000000,2014-09-30,2015-10-01,201501,60


In [18]:
rastro_contratos.loc[rastro_contratos['id_antigo'] == '10000000']

,id_antigo,id_novo,data_evento
1023,10000000,10004583,2015-12-01


# <font color='red' style='font-size: 40px;'> Exemplo Rolagem </font>
<hr style='border: 2px solid red;'>

https://www.kaggle.com/datasets/adilshamim8/credit-risk-benchmark-dataset

# <font color='red' style='font-size: 40px;'> Exemplo Inferência de Negados </font>
<hr style='border: 2px solid red;'>

https://python.plainenglish.io/a-project-on-reject-inference-94a6858bc821
https://www.kaggle.com/code/shraddhacodes/credit-card-default-reject-inference-project

# <font color='green' style='font-size: 30px;'> 1.1) Teste 1 </font>
<hr style='border: 2px solid green;'>

# <font color='red' style='font-size: 40px;'> Exemplo Regressão Logística + Binning + Scorecard </font>
<hr style='border: 2px solid red;'>


https://www.kaggle.com/datasets/s3r1alsh0ck/finance-dataset-for-credit-risk-fraud-detection

# <font color='green' style='font-size: 30px;'> 1.1) Teste 1 </font>
<hr style='border: 2px solid green;'>